In [1]:
import os, sys, glob
import subprocess
import tempfile
from tqdm import tqdm
import copy
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray
from pyproj import Proj, Transformer, Geod
from rioxarray import merge
from osgeo import gdal
from joblib import Parallel, delayed
from oggm import utils
import gzip
import json
import topojson
import shapely
from shapely.geometry import Polygon, MultiPolygon
from shapely.wkt import loads, dumps
from pathlib import Path

In [2]:
# Path to the folder containing fetch_glacier_metadata.py
iceboost_path = Path("..") / "iceboost"
sys.path.insert(0, str(iceboost_path))

# Now you can import those two functions
from fetch_glacier_metadata import get_rgi_products, add_regional_features

In [7]:
# RGI
version = '62'
#version = '70G'
if version == '62':
    name_rgi_id = 'RGIId'
    name_column_area = 'Area'
    name_column_name = 'Name'
    name_column_zmin = 'Zmin'
    name_column_zmax = 'Zmax'
    name_column_zmed = 'Zmed'
elif version == '70G':
    name_rgi_id = 'rgi_id'
    name_column_area = 'area_km2'
    name_column_name = 'glac_name'
    name_column_zmin = 'zmin_m'
    name_column_zmax = 'zmax_m'
    name_column_zmed = 'zmed_m'
else: raise ValueError('Version is wrong. Ciao')
OUT_PATH = f"/home/maffe/PycharmProjects/iceboost_webapp"

In [8]:
list_gdfs = []

for rgi in range(1,20):

    FILE_SHP_RGI = utils.get_rgi_region_file(region=f"{rgi:02d}", version=version)
    print(rgi, FILE_SHP_RGI)

    # NEW
    gdf, _ = get_rgi_products(region=rgi,
                                    version=version,
                                    add_glacier_geom_file=None,
                                    add_glacier_intersects_geom_file=None)
    gdf = add_regional_features(gdf)
    #print(list(gdf))
    #print(gdf.shape)
    #input('wait')

    # OLD
    #gdf = gpd.read_file(FILE_SHP_RGI)
    #print(list(gdf))
    #print(gdf.shape)


    #gdf = gdf[[name_rgi_id, name_column_name, name_column_area, 'geometry']]
    gdf = gdf[[name_rgi_id, name_column_name, 'area', 'perimeter', 'lmax', name_column_zmin, name_column_zmax, name_column_zmed, 'geometry']]

    gdf.rename(columns={
        name_rgi_id: 'rgi_id',
        name_column_name: 'name',
        #name_column_area: 'area'
        name_column_zmin: 'zmin',
        name_column_zmax: 'zmax',
        name_column_zmed: 'zmed',
    }, inplace=True)
    
    #if name_rgi_id == 'RGIId':
    #    gdf.rename(columns={'RGIId': 'rgi_id'}, inplace=True)

    gdf_rounded = gdf.copy()

    gdf_rounded['geometry'] = gdf_rounded['geometry'].apply(lambda geom: shapely.set_precision(geom, 0.00001))

    list_gdfs.append(gdf_rounded)

#input('out of loop')

print('Combining geodataframes')
combined_gdf = pd.concat(list_gdfs, ignore_index=True)
print('Done.')

print('Creating geojson')
geojson_file = f"{OUT_PATH}/all_geometries_rgi{version}_cleaned.geojson"
print(OUT_PATH)
print(geojson_file)

combined_gdf.to_file(geojson_file, driver='GeoJSON')

#tippecanoe_path_out = f"/home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI{version}/geometries/"
tippecanoe_path_out = f"/home/maffe/PycharmProjects/iceboost_webapp/"
tippecanoe_file_out = f"geoms_rgi{version}.mbtiles"
tippecanoe_out_layer_name = f"geoms_rgi{version}"
tippecanoe_out = tippecanoe_path_out + tippecanoe_file_out

# Construct the Tippecanoe command
tippecanoe_command = f"tippecanoe -o {tippecanoe_out} -l {tippecanoe_out_layer_name} -zg {geojson_file} --force --preserve-input-order"

# Run Tippecanoe command via subprocess
subprocess.run(tippecanoe_command, shell=True, check=True)

# remove geojson_file
os.remove(geojson_file)
print("Deleted: ", geojson_file)

1 /home/maffe/OGGM/rgi/RGIV62/01_rgi62_Alaska/01_rgi62_Alaska.shp
2 /home/maffe/OGGM/rgi/RGIV62/02_rgi62_WesternCanadaUS/02_rgi62_WesternCanadaUS.shp
3 /home/maffe/OGGM/rgi/RGIV62/03_rgi62_ArcticCanadaNorth/03_rgi62_ArcticCanadaNorth.shp
4 /home/maffe/OGGM/rgi/RGIV62/04_rgi62_ArcticCanadaSouth/04_rgi62_ArcticCanadaSouth.shp
5 /home/maffe/OGGM/rgi/RGIV62/05_rgi62_GreenlandPeriphery/05_rgi62_GreenlandPeriphery.shp
6 /home/maffe/OGGM/rgi/RGIV62/06_rgi62_Iceland/06_rgi62_Iceland.shp
7 /home/maffe/OGGM/rgi/RGIV62/07_rgi62_Svalbard/07_rgi62_Svalbard.shp
8 /home/maffe/OGGM/rgi/RGIV62/08_rgi62_Scandinavia/08_rgi62_Scandinavia.shp
9 /home/maffe/OGGM/rgi/RGIV62/09_rgi62_RussianArctic/09_rgi62_RussianArctic.shp
10 /home/maffe/OGGM/rgi/RGIV62/10_rgi62_NorthAsia/10_rgi62_NorthAsia.shp
11 /home/maffe/OGGM/rgi/RGIV62/11_rgi62_CentralEurope/11_rgi62_CentralEurope.shp
12 /home/maffe/OGGM/rgi/RGIV62/12_rgi62_CaucasusMiddleEast/12_rgi62_CaucasusMiddleEast.shp
13 /home/maffe/OGGM/rgi/RGIV62/13_rgi62_Centr

216502 features, 188031090 bytes of geometry, 8845162 bytes of separate metadata, 16849494 bytes of string pool
Choosing a maxzoom of -z5 for features about 9827 feet (2996 meters) apart
Choosing a maxzoom of -z10 for resolution of about 347 feet (106 meters) within features
tile 1/0/0 size is 511344 with detail 12, >500000    
tile 2/1/0 size is 697088 with detail 12, >500000    
tile 3/5/3 size is 569384 with detail 12, >500000    
tile 4/1/4 size is 698437 with detail 12, >500000    
tile 4/11/6 size is 1354497 with detail 12, >500000    
tile 4/11/6 size is 551532 with detail 11, >500000    
tile 5/23/13 size is 512598 with detail 12, >500000    
tile 5/3/9 size is 572806 with detail 12, >500000    
tile 5/22/12 size is 1695221 with detail 12, >500000    
tile 5/22/12 size is 883623 with detail 11, >500000    
tile 6/45/24 size is 588153 with detail 12, >500000    
tile 6/44/24 size is 676923 with detail 12, >500000    
tile 6/45/25 size is 1178921 with detail 12, >500000    
tile 

Deleted:  /home/maffe/PycharmProjects/iceboost_webapp/all_geometries_rgi62_cleaned.geojson


In [15]:
# Antarctic Peninsula

In [15]:
from shapely.validation import make_valid
from shapely.validation import explain_validity

#PENINSULA_PATH = f"/media/maffe/nvme/Antarctic_peninsula_geometries/final_product/"
PENINSULA_PATH = f"/media/maffe/nvme/antarctic_peninsula_from_bedmachinev4/"

peninsula = gpd.read_file(PENINSULA_PATH+"antarctic_peninsula.shp")

if not peninsula.crs == "EPSG:4326":
    peninsula = peninsula.to_crs("EPSG:4326")

peninsula = peninsula[['geometry']]

# correct some geometries
peninsula['geometry'] = peninsula['geometry'].apply(
    lambda geom: make_valid(geom) if not geom.is_valid else geom
)

peninsula_rounded = peninsula.copy()

#n_invalid_before = (~peninsula_rounded.is_valid).sum()
#invalid_geoms = peninsula_rounded.loc[~peninsula_rounded.is_valid, 'geometry']
#reasons = invalid_geoms.apply(explain_validity)
#for idx, reason in reasons.items():
#    print(f"Index {idx}: {reason}")
#print(n_invalid_before)
#input('wait')

peninsula_rounded['geometry'] = peninsula_rounded['geometry'].apply(lambda geom: shapely.set_precision(geom, 0.00001))

print('Creating geojson')
geojson_file = f"{PENINSULA_PATH}peninsula_cleaned.geojson"
print(PENINSULA_PATH)
print(geojson_file)

peninsula_rounded.to_file(geojson_file, driver='GeoJSON')

tippecanoe_path_out = f"{PENINSULA_PATH}"
tippecanoe_file_out = f"peninsula.mbtiles"
tippecanoe_out_layer_name = f"peninsula"
tippecanoe_out = tippecanoe_path_out + tippecanoe_file_out

# Construct the Tippecanoe command
tippecanoe_command = f"tippecanoe -o {tippecanoe_out} -l {tippecanoe_out_layer_name} -zg {geojson_file} --force"

# Run Tippecanoe command via subprocess
subprocess.run(tippecanoe_command, shell=True, check=True)

# remove geojson_file
os.remove(geojson_file)
print("Deleted: ", geojson_file)

Creating geojson
/media/maffe/nvme/antarctic_peninsula_from_bedmachinev4/
/media/maffe/nvme/antarctic_peninsula_from_bedmachinev4/peninsula_cleaned.geojson
Deleted:  /media/maffe/nvme/antarctic_peninsula_from_bedmachinev4/peninsula_cleaned.geojson


50 features, 546985 bytes of geometry, 66 bytes of separate metadata, 0 bytes of string pool
Choosing a maxzoom of -z0 for features about 1086978 feet (331311 meters) apart
Choosing a maxzoom of -z5 for resolution of about 8538 feet (2602 meters) within features
  95.8%  5/10/24  
